In [1]:
import cv2 # computer vision tasks such as image and video processing, object detection
import time # measuring time intervals, delaying execution, and working with timestamps
import numpy as np # numerical operations

In [2]:
# Preparation for writing the ouput video

fourcc = cv2.VideoWriter_fourcc(*'XVID')  # for encoding or decoding a video file
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480)) # encode video frames using fourcc, frame rate of 20 frames per second and a frame size 640x480 pixels and save as 'output.avi'

#reading from the webcam

cap = cv2.VideoCapture(0) # capture video frames from a device's camera

# Allow the system to sleep for 3 seconds before the webcam starts

time.sleep(3) # Suspends execution for 3 seconds before continueing next line

In [ ]:
count = 0
background = 0

# Capture the background in range of 60
for i in range(60):
    ret, background = cap.read() # Read a single frame from the camera
background = np.flip(background, axis=1) # flips the background array horizontally using NumPy's flip() function, reducing lateral inversion, provides mirror image
while (cap.isOpened()): # check if a VideoCapture object is open and able to capture frames from a video source
    ret, img = cap.read()
    if not ret: # ret is a Boolean value that indicates whether the frame was successfully read
        break
    count += 1
    img = np.flip(img, axis=1)

    # Convert the color space from BGR to HSV (Hue, Saturation, Value)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Generat masks to detect red color
    lower_red = np.array([0, 120, 50]) # lower threshold values for the red color range in the HSV color space
    upper_red = np.array([10, 255,255]) #  upper threshold values for the red color range in the HSV color space
    mask1 = cv2.inRange(hsv, lower_red, upper_red) # creates a binary mask 

    lower_red = np.array([170, 120, 70])
    upper_red = np.array([180, 255, 255])
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    mask1 = mask1 + mask2

    # Open and Dilate the mask image
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8)) #  morphological opening to the mask1 binary mask using a 3x3 kernel
    mask1 = cv2.morphologyEx(mask1, cv2.MORPH_DILATE, np.ones((3, 3), np.uint8)) # Morphological dilation expands the white regions in the binary mask, making them larger and more connected

    # Create an inverted mask to segment out the red color from the frame
    mask2 = cv2.bitwise_not(mask1) # it converts white pixels to black and black pixels to white

    # Segment the red color part out of the frame using bitwise and with the inverted mask
    res1 = cv2.bitwise_and(img, img, mask=mask2)

    # Create image showing static background frame pixels only for the masked region
    res2 = cv2.bitwise_and(background, background, mask=mask1)

    # Generating the final output and writing
    finalOutput = cv2.addWeighted(res1, 1, res2, 1, 0) # combines two images res1 and res2
    out.write(finalOutput)
    cv2.imshow("magic", finalOutput)
    cv2.waitKey(1)
cap.release()
out.release()
cv2.destroyAllWindows() # close and destroy all the windows